<div style="font-size: 200%; font-weight: bold; color: maroon;">Machine Learning en SparkR - modelos distribuidos</div>
<!-- v. 1.1 - Paulo Villegas, 2017 -->

Adaptado de un [ejemplo de SparkR](https://github.com/apache/spark/blob/master/examples/src/main/r/ml/ml.R).

Vamos a entrenar varios modelos distintos (usando diferentes hiperparámetros) en paralelo, mediante la función `spark.lapply`. Nuestro clasificador será un SVM (del paquete [e1701](https://cran.r-project.org/web/packages/e1071/index.html)). Trataremos de clasificar las 3 especies de IRIS en función de sus _features_ 

In [ ]:
# Load SparkR and create a session
library(SparkR)
spark <- sparkR.session( "local[*]" );

Usamos de nuevo el típico dataset de IRIS. En este caso no usamos SparkDataFrames (como consecuencia, el dataset _debe_ caber dentro de un nodo ejecutor de Spark)

In [ ]:
# El rango de hiperparámetros de nuestro SVM que vamos a probar
svm.costs <- exp( seq(from=log(1), to=log(1000), length.out=5) )

In [ ]:
svm.costs

In [ ]:
# La función de entrenamiento
# Nótese que usamos el dataset [iris] en la función, por lo que queda incluido en la closure que
# el ejecutor de Spark mandará a todos los nodos
train <- function(cost) {
  stopifnot( requireNamespace("e1071", quietly = TRUE) )
  model <- e1071::svm( Species ~ ., data = iris, cost = cost )
}

In [ ]:
# Entrenamos
model.list <- spark.lapply(svm.costs, train)

In [ ]:
# Veamos los resultados
for( model in model.list) {
    cat("\nMODEL =", model$cost)
    pred <- fitted( model, iris )
    print( table(pred, iris$Species) )
}